**Business Problem:**
- You have been hired to produce a MySQL database on Movies from a subset of IMDB's publicly available dataset. This database will be used to analyze what makes a movie successful and will provide recommendations to the stakeholder on how to make a successful movie.

    Lets download the official IMDB data for the requested tables, filtering out unnecessary data, and saving the filtered tables as gzip-compressed csv files (".csv.gz") in  repository.

**Data**
- IMDB Provides Several Files with varied information for Movies, TV Shows, Made for TV Movies, etc.

- Overview/Data Dictionary: https://www.imdb.com/interfaces/
- Downloads page: https://datasets.imdbws.com/

### Cleaning the data for analysis

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
## Making "Data" Folder to save IMDB files
os.makedirs("Data/",exist_ok=True)
os.listdir("Data/")

['title_basics_cleaned.csv.gz',
 'title_ratings_cleaned.csv.gz',
 'tmdb_api_results_2000.json',
 'final_tmdb_data_2000.csv.gz',
 'tmdb_api_results_2001.json',
 'final_tmdb_data_2002.csv.gz',
 'tmdb_api_results_2002.json',
 'final_tmdb_data_2001.csv.gz',
 'title_akas_cleaned.csv.gz']

In [3]:
## title basics 
url_title_basics = 'https://datasets.imdbws.com/title.basics.tsv.gz'
basics = pd.read_csv(url_title_basics, sep='\t',low_memory=False)
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
10534788,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2009,\N,\N,"Action,Drama,Family"
10534789,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
10534790,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
10534791,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [4]:
## title ratings
url_title_ratings ="https://datasets.imdbws.com/title.ratings.tsv.gz"
ratings = pd.read_csv(url_title_ratings,sep='\t',low_memory=False)
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,2022
1,tt0000002,5.7,271
2,tt0000003,6.5,1952
3,tt0000004,5.4,178
4,tt0000005,6.2,2722
...,...,...,...
1399980,tt9916730,7.6,11
1399981,tt9916766,7.1,23
1399982,tt9916778,7.2,36
1399983,tt9916840,8.8,6


In [5]:
## title AKAs
url_title_akas ="https://datasets.imdbws.com/title.akas.tsv.gz"
akas = pd.read_csv(url_title_akas,sep='\t',low_memory=False)
akas

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0
...,...,...,...,...,...,...,...,...
38621227,tt9916852,5,Episódio #3.20,PT,pt,\N,\N,0
38621228,tt9916852,6,Episodio #3.20,IT,it,\N,\N,0
38621229,tt9916852,7,एपिसोड #3.20,IN,hi,\N,\N,0
38621230,tt9916856,1,The Wind,DE,\N,imdbDisplay,\N,0


#### Cleaning and preprocessing Title AKAS file

In [6]:
# Replace "\N" with np.nan (if any)
akas.replace({'\\N':np.nan},inplace=True)
akas.isna().sum()

titleId                   0
ordering                  0
title                     5
region              1919226
language            6923572
types              32806403
attributes         38347702
isOriginalTitle        2077
dtype: int64

In [7]:
## The AKAs file has the information on country where it released and language
akas = akas[(akas['region'] == 'US')]
akas

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0
...,...,...,...,...,...,...,...,...
38620758,tt9916560,1,March of Dimes Presents: Once Upon a Dime,US,NaN,imdbDisplay,NaN,0
38620828,tt9916620,1,The Copeland Case,US,NaN,imdbDisplay,NaN,0
38620917,tt9916702,1,Loving London: The Playground,US,NaN,NaN,NaN,0
38620960,tt9916756,1,Pretty Pretty Black Girl,US,NaN,imdbDisplay,NaN,0


#### Cleaning and Preprocessing BASICS

In [8]:
## check nulls
basics.isna().sum()

tconst             0
titleType          0
primaryTitle      11
originalTitle     11
isAdult            0
startYear          0
endYear            0
runtimeMinutes     0
genres            18
dtype: int64

In [9]:
## Replace "\N" with np.nan
basics.replace({'\\N':np.nan},inplace=True)
basics.isna().sum()

tconst                   0
titleType                0
primaryTitle            11
originalTitle           11
isAdult                  1
startYear          1394944
endYear           10416439
runtimeMinutes     7308675
genres              468711
dtype: int64

In [10]:
## Eliminate movies that are null for runtimeMinute, genres and startYear
basics = basics.dropna(subset=['runtimeMinutes','genres','startYear'])
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            1
originalTitle           1
isAdult                 0
startYear               0
endYear           2891215
runtimeMinutes          0
genres                  0
dtype: int64

In [11]:
## keep only titleType==Movie
basics = basics.loc[ basics['titleType']=='movie']
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,NaN,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
...,...,...,...,...,...,...,...,...,...
10534643,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019,NaN,123,Drama
10534684,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,NaN,57,Documentary
10534711,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,NaN,100,Documentary
10534733,tt9916730,movie,6 Gunn,6 Gunn,0,2017,NaN,116,Drama


In [12]:
## Eliminate movies that include  "Documentary" in genre 
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
930,tt0000941,movie,Locura de amor,Locura de amor,0,1909,NaN,45,Drama


In [13]:
## keep startYear 2000-2022

# Convert startYear as float before filtering
basics['startYear'] = basics['startYear'].astype(float)

basics = basics[(basics['startYear']>=2000)&(basics['startYear']<2022)]
basics

/var/folders/td/cybzgn316hq8r5cn05lb0y2m0000gn/T/ipykernel_48792/1957374321.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  basics['startYear'] = basics['startYear'].astype(float)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34798,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61108,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67660,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
80545,tt0082328,movie,Embodiment of Evil,Encarnação do Demônio,0,2008.0,NaN,94,Horror
86787,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
10534475,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"
10534514,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020.0,NaN,84,Thriller
10534559,tt9916362,movie,Coven,Akelarre,0,2020.0,NaN,92,"Drama,History"
10534643,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019.0,NaN,123,Drama


In [14]:
basics.describe()

,startYear
count,139988.000000
mean,2012.771930
std,5.750282
min,2000.000000
25%,2009.000000
50%,2014.000000
75%,2018.000000
max,2021.000000


#### Cleaning and Preprocessing Title Ratings

In [15]:
# Replace "\N" with np.nan (if any)
ratings.replace({'\\N':np.nan},inplace=True)
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

- Filtering Out Non-US Movies
    - Now that both AKAs and basics have been processed, we can remove any movie ids from title basics that are NOT in the US-filtered AKAs dataframe.

In [16]:
#Removing Non-US From Title Basics
keepers = basics['tconst'].isin(akas['titleId'])
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34798,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61108,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67660,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
80545,tt0082328,movie,Embodiment of Evil,Encarnação do Demônio,0,2008.0,NaN,94,Horror
86787,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
10533932,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
10534326,tt9915872,movie,The Last White Witch,Boku no kanojo wa mahoutsukai,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
10534466,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
10534475,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


In [17]:
# Removing Non - US from Ratings
keepers = ratings['tconst'].isin(akas['titleId'])
ratings = ratings[keepers]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,2022
1,tt0000002,5.7,271
4,tt0000005,6.2,2722
5,tt0000006,5.0,184
6,tt0000007,5.4,848
...,...,...,...
1399945,tt9916200,8.1,240
1399946,tt9916204,8.1,276
1399953,tt9916348,8.3,18
1399954,tt9916362,6.4,5696


#### Saving Final Files

In [18]:
## Saving and immediately loading (to verify)
basics.to_csv('testData/title_basics_cleaned.csv.gz',compression='gzip',index=False)
basics = pd.read_csv('testData/title_basics_cleaned.csv.gz')
basics.info()
basics.head()

OSError: Cannot save file into a non-existent directory: 'testData'

In [ ]:
ratings.to_csv('testData/title_ratings_cleaned.csv.gz',compression='gzip',index=False)
ratings = pd.read_csv('testData/title_ratings_cleaned.csv.gz')
ratings.info()
ratings.head()

In [ ]:
akas.to_csv('testData/title_akas_cleaned.csv.gz',compression='gzip',index=False)
akas = pd.read_csv('testData/title_akas_cleaned.csv.gz')
akas.info()
akas.head()

In [ ]:
# optional Checking for files in Data Folder
os.listdir("Data/")